In [ ]:
!curl https://raw.githubusercontent.com/APSV-UPM/BusinessIntelligence/main/data/data_clean.csv > data.csv

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd

Let's load our well known dataset of train events:

In [ ]:
df = pd.read_csv("data.csv").dropna()
df.date = pd.to_datetime(df.date)

We are going to transform this train event dataset into a table where each row contains information for a single train trip. Each row will include the number of packages transported, the train used, and the trip duration.

In [ ]:
t = df.groupby("trip_id").agg({"train_name":"first", "date":{'min', 'max'}, "cargo":'max', "fuel_level":{"min", "max"}, "cargo_type":"last"})
t.columns.map("_".join)

In [ ]:
trip_summary = df.groupby("trip_id").agg({"train_name":"first", "date":{'min', 'max'}, "cargo":'max', "fuel_level":{"min", "max"}, "cargo_type":"last"})
trip_summary.columns = trip_summary.columns.map("_".join)
trip_summary.columns = trip_summary.rename(columns={"date_min":"start_date", "date_max":"end_date", "train_name_first":"train_name", "cargo_max":"cargo", "cargo_type_last":"cargo_type"}).columns
trip_summary.reset_index(inplace=True, drop=True)
trip_summary["trip_duration"] = (trip_summary.end_date - trip_summary.start_date).dt.total_seconds()
trip_summary["fuel_consumed"] = trip_summary.fuel_level_max - trip_summary.fuel_level_min
trip_summary

Most ML algorithms work with numerical values; therefore, we must convert columns that contain text (categorical variables) into numbers. The simplest way to do this is to use a LabelEncoder, which assigns an integer to each category it finds.

In [ ]:
from sklearn import preprocessing
train_name_encoder = preprocessing.LabelEncoder().fit(trip_summary.train_name.unique())

for i in range(len(train_name_encoder.classes_)):
    print(f"""{train_name_encoder.classes_[i]} encoded as {i}""")
trip_summary["train_name_encoded"] = pd.Series(train_name_encoder.transform(trip_summary.train_name.values), index = trip_summary.index)

Now we need to do the same for the `cargo_type` column. Use the next cell to do it, and don't forget to save the encoder for later use.

In [ ]:
# Write your code here

### Train and test
The main idea behind machine learning is that the algorithm observes a training set of samples, which should be representative of the data we're working with. From this training data, the algorithm learns patterns that describe the data and generates a model that can reproduce the observed behavior. Then, using a *different* set of samples (the test set), we evaluate how accurate the model is by measuring the difference between the model's outputs and the correct answers. It is very important not to use the same data for training and testing.

Let's see an example.
We will use a popular dataset in ML called the Iris dataset. This dataset contains measurements of flowers and the flower species (for more info, see https://archive.ics.uci.edu/ml/datasets/iris). As we will see, the task in this example is classification.
The dataset consists of observations with four numeric features and one categorical target. The model's task is to predict the target class of a sample from its numeric features.


In [ ]:
# This example is based on one example from the scikit learn documentation 
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html
from sklearn import neighbors, datasets
from sklearn import model_selection

x_feature = 0
y_feature = 1

n_neighbors = 5

iris = datasets.load_iris()

# In this notebook, we will always use x as the feature vector and y as the target
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.3)

clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(x_train, y_train)
y_out = clf.predict(x_test)

fig,ax = plt.subplots(figsize=(9, 8))

ax.scatter(x_train[:, x_feature], x_train[:, y_feature], c=y_train, cmap="plasma", edgecolors='k')
ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_out, cmap="plasma", marker = "x")
ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_test, cmap="plasma", alpha=0.3, marker = "o", s=200, edgecolors='k')
ax.scatter(x_test[y_out != y_test, x_feature], x_test[y_out != y_test, y_feature], facecolors='none', edgecolors='r', s=200)


ax.set_title("3-Class classification (k = %i)" % (n_neighbors))
plt.show()

print("Model accuracy: {:.2f}%".format(np.average(y_out==y_test)*100))

Although the dataset has four dimensions, the plot shows only two, which is easier to visualize. The colors represent different flower types. Dots are training data and crosses are test data. The background color of each test sample indicates the true class, and a sample is surrounded by a red circle if it was misclassified. Accuracy is the proportion of correct classifications made by the model.

### K-Fold Cross-Validation
Ensuring that the partition of the data used for training is representative can be difficult. For example, the training split might not contain samples of one class, or the test split might be particularly easy to classify.

Cross-validation is a set of techniques that addresses this problem by performing several train/test runs using different partitions each time. The most basic and widely used method is K-Fold cross-validation.
In K-Fold, we choose a number k (usually between 5 and 10) and divide the dataset into k folds. We train the model k times; in each iteration we use k-1 folds for training and the remaining fold for testing. The final evaluation is the average of the evaluations from each iteration.

In [ ]:
from sklearn.model_selection import KFold
k = 3
kf = KFold(n_splits=k,shuffle=True)
accuracies = []
i=0
for train, test in kf.split(X): # train and test are the indices of the samples that will be used in each set
    i+=1
    x_train, x_test, y_train, y_test = X[train], X[test], y[train], y[test]
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    clf.fit(x_train, y_train)
    y_out = clf.predict(x_test)


    fig,ax = plt.subplots(figsize=(9, 8))
    ax.scatter(x_train[:, x_feature], x_train[:, y_feature], c=y_train, cmap="plasma", edgecolors='k')
    ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_out, cmap="plasma", marker = "x")
    ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_test, cmap="plasma", alpha=0.3, marker = "o", s=200, edgecolors='k')
    ax.scatter(x_test[y_out != y_test, x_feature], x_test[y_out != y_test, y_feature], facecolors='none', edgecolors='r', s=200)
    ax.set_title("3-Class classification (k = %i) iteration %i" % (n_neighbors,i))
    plt.show()
    accuracy = np.average(y_out==y_test)*100
    accuracies.append(accuracy)
    print("Iteration {}: Model accuracy: {:.2f}%".format(i,accuracy))
    print(y_test)
    print(y_out)
print("Global accuracy: {:.2f}%".format(np.average(accuracies)))

## Regression
Regression is a type of ML task where the objective is to predict a numeric value from input features. For example, predicting the number of products that will be sold based on previous sales, or estimating the probability that the stock market will rise tomorrow.

The simplest example is finding the relationship between two variables.

In [ ]:
from sklearn import datasets
import math

x, y = datasets.make_regression(n_samples=30, n_features=1, noise=10, random_state=42)
# try to modify the relation between x and y or play with the noise value
#y = y**2
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
plt.show()

Problems in the real world are more complicated, the target (variable we want to predict) usually depends on more variables (features).

### Some popular algorithms
There are many algorithms that can be used for regression. Each has advantages and disadvantages depending on the problem; discussing them in depth is beyond the scope of this course. We'll try them with basic configurations and keep the one that best fits our case.

#### Linear regression
This is the simplest regression model. The algorithm tries to minimize the error for a function like $Y = aX + b$, where $Y$ is the target and $X$ is the input/features.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x,y)
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
ax.plot(x,model.predict(x), "r")
plt.show()

#### Decision Tree
This model is based on a tree structure in which each node represents a decision and each leaf an output. https://en.wikipedia.org/wiki/Decision_tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
x_test= np.linspace(x.min(),x.max(), 100)[:, np.newaxis]

model = DecisionTreeRegressor().fit(x,y)
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
ax.plot(x_test ,model.predict(x_test),"r")

plt.show()

#### Random Forest
This model is an ensemble method composed of multiple decision trees. Each tree predicts an output, and the random forest's output is the average (or majority vote for classification) of those tree outputs.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

#### K Nearest Neighbors
Given an input, the output of this algorithm will be the average of the $k$ nearest points in the training set.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=5).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

#### Perceptron
This is a simple artificial neural network (see https://en.wikipedia.org/wiki/Artificial_neural_network). Unlike some of the models above, neural networks often behave like a "black box" and it's hard to inspect exactly how they compute outputs.

Don't worry if you get a warning when running this model.

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(max_iter=50000).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

### How to evaluate a regression model
We have trained five models and we can expect some to perform better than others, but we need a formal way to evaluate them. Several metrics measure the error between the model's output and the true values on the test data; the lower the error, the better the model. Here are some common metrics:

1. Max error: $max(| y_i - \hat{y}_i |)$

2. Mean absolute error or MAE: $\frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|$

3. Mean squared error or MSE: $\frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2$

Don't worry about the math, scikit-learn will compute these metrics for us.

Remember about K-Folds Cross Validation when evaluating the models.

In [ ]:
from sklearn.metrics import  mean_absolute_error, mean_squared_error, max_error
models = {"Linear Regression":LinearRegression(), \
          "Decision Tree": DecisionTreeRegressor(), \
          "Random Forest": RandomForestRegressor(n_estimators=100), \
          "Nearest Neighbors": KNeighborsRegressor(n_neighbors=5), \
          "Perceptron": MLPRegressor(max_iter=2000)}

x, y = datasets.make_regression(n_samples=30, n_features=1, noise=10)
k = 3
kf = KFold(n_splits=k,shuffle=True)
for name in models:
    print("Start training models of {}".format(name))
    i = 0
    me_global = []
    mae_global = []
    mse_global = []
    for train, test in kf.split(x):
        i+=1
        x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
        model = models[name].fit(x_train,y_train)
        y_out = model.predict(x_test)
        # We have to provide the real value of the target and the model's output
        me = max_error(y_test, y_out)
        mae = mean_absolute_error(y_test, y_out)
        mse = mean_squared_error(y_test, y_out)
        me_global.append(me)
        mae_global.append(mae)
        mse_global.append(mse)
        print("Iteration {}: me={:.4f} mae={:.4f} mse={:.4f}".format(i,me, mae, mse))
    print("Global: me={:.4f} mae={:.4f} mse={:.4f}\n".format(np.average(me_global), np.average(mae_global), np.average(mse_global)))

### Try it
Now it's your turn to create a prediction model. After reading the datasets, we have transformed and prepared them in a format suitable for the algorithms shown above.

Let's try to build a regression model to predict the fuel consumed during a trip. If we want to use this model as a practical prediction tool, the features we use must be available before the trip starts. That means we cannot use final fuel level or trip duration as features, since we want to predict the target before the trip begins.
Train several regression models using different algorithms and keep the best one. Then answer the following questions:

- What features did you use to train the model?
- Which algorithm performed best?
- What metric did you use to select the best model?

In [ ]:
# Code here!

# You can use a similar approach to the one used in the previous cells to create your regression model.
# To extract the features and target from the trip_summary dataframe, follow the example below:
x = trip_summary[["train_name_encoded", "cargo"]].values # this is using train_name and cargo as features
y = trip_summary.fuel_consumed.values # this is using fuel_consumed as target


In [ ]:
# To evaluate this task, you will be asked to run your model in a given situation. 
# For example, the train UDAXI has a scheduled trip on 2025-01-23 at 10:00:00, its cargo will be 50 packages of electronics and it will be loaded with 800 units of fuel. 
# How much fuel will it consume during the trip?

# Suppose that your model is called "model" and your input is a vector of [train_name, cargo]. Then, you will have to run the following code:
# transform the name of the train to the encoded value: encoded_name = train_name_encoder.transform(["UDAXI"])[0]
# make the prediction: model.predict([[encoded_name,50]])
# the last line will return the fuel consumption

## Classification
Another common ML task is classification; in this case the target is a class label or categorical variable, as with the Iris dataset.

When discussing classification tasks there are two main cases: binary classification (target is positive or negative) and multiclass classification (target is one of several categories).

In [ ]:
wine = datasets.load_wine()
x = wine.data[:, 0:2]  # we only take the first two features for easy
y = wine.target
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)
plt.show()

To better understanding of the models output, we can visualize the decision boundaries. A decision boundary is a region in the feature space where the model changes its predicted class. To represent this, we can create a grid of points that cover the feature space and use the model to predict the class for each point. Then we can color each point according to its predicted class, which will show us how the model divides the feature space.

In [ ]:
# For decision regions
x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))


### Some popular models
As with regression, there are many algorithms for classification. Many algorithms have both regression and classification variants.

The following plots show decision regions — the areas in feature space where the model assigns a given class.

#### Logistic regression
Don't be misled by the name: Logistic Regression is used for classification. The model estimates the probability that a sample belongs to a given class using the logistic function.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class="auto", solver="lbfgs").fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier().fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(max_iter=2000).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

### How to evaluate a classification model
For classification, the output is a class label, so we do not measure error the same way as in regression. Instead, we compute ratios such as accuracy, precision, and recall. The most intuitive measure is accuracy — the fraction of correct predictions out of all predictions. However, accuracy can be misleading in many practical situations.

### Confusion matrix
The confusion matrix is a primary tool for evaluating classification models (https://en.wikipedia.org/wiki/Confusion_matrix). It is defined as:

"By definition a confusion matrix $C$ is such that $C_{i,j}$ is equal to the number of observations known to be in group $i$ but predicted to be in group $j$."

In other words, rows represent the true classes and columns represent the predicted classes. The confusion matrix shows how many samples were correctly classified and which classes are commonly confused.

In the binary classification case, the matrix has 2 rows and 2 columns, and each cell has a conventional name.

Some metrics derived from the confusion matrix include:
1. Precision: the ratio of true positives over predicted positives $\frac{TP}{TP+FP}$
2. Accuracy: the ratio of correctly predicted samples over the total $\frac{TP+TN}{TP+FP+TN+FN}$
3. Recall (Sensitivity): the ratio of true positives over actual positives $\frac{TP}{TP+FN}$
4. F1 score: the harmonic mean of precision and recall $\frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} = \frac{2TP}{2TP + FP + FN}$

More complex metrics exist, but for this course the F1 score is sufficient. Again, scikit-learn provides functions to compute these metrics.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# x = wine.data # uncomment this line to use all features

models = {"Logistic Regression":LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=2000), \
          "Decision Tree": DecisionTreeClassifier(), \
          "Random Forest": RandomForestClassifier(n_estimators=100), \
          "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5), \
          "Perceptron": MLPClassifier(max_iter=2000)}

for name in models:
    print("Start training models of {}".format(name))
    i = 0
    f1_global = []
    for train, test in kf.split(x):
        i+=1
        x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
        model = models[name].fit(x_train,y_train)
        y_out = model.predict(x_test)
        # We have to provide the real value of the target and the model's output
        f1_global.append(f1_score(y_test, y_out, average="weighted", zero_division=0))
        print(confusion_matrix(y_test, y_out))
        print(classification_report(y_test, y_out))
    print("Global: f1={:.4f}\n".format(np.average(f1_global)))

### Try it

Imagine that the packages that arrive to the last station are not labeled and we cannot identify what type of cargo they contain. Let's build a classification model to help us to guess the correct cargo type. In this case, we will use the model at the end of the trip, so we can use all available features except the cargo type, which is the target. As before, train several classification models using different algorithms and keep the best one. Then answer the following questions:

- Which algorithm performs best?
- What is the average F1 score for that algorithm?

Some algorithms provide estimates of feature importance. This is useful to understand which features matter most. Tree-based models (Decision Tree and Random Forest) expose this via the attribute `feature_importances_`. The higher the value, the more important the feature. In this case, which feature is most important?

In [ ]:
# Code here!

In [ ]:
# To evaluate this task, you will be asked to run your model in a given situation.
# For example, the train HHEXD arrives to the last station. It has transported 40 packages. 
# The trip started at 2025-01-15 08:00:00 with 800 units of fuel and ended at 2025-01-15 12:00:00 with 350 units of fuel.
# What type of cargo is it carrying?